# Assignment 3: Predictive analysis - Part 2: Classification
## Group 105
- Natasa Bolic (300241734)
- Brent Palmer (300193610)
## Imports

In [1]:
# imports
import pandas as pd

## Introduction

## Dataset Description

**Url:** https://www.kaggle.com/datasets/blastchar/telco-customer-churn <br>
**Name:** Telco Customer Churn <br>
**Author:**  <br>
**Purpose:** <br>
**Shape:** There are X rows and Y columns. (X, Y)<br>
**Features:** 
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():
- `` ():

## Loading Dataset and Basic Exploration

In [ ]:
# Read in the dataset from a public repository
url = "https://raw.githubusercontent.com/Natasa127/CSI4142-A2/main/dirty_cafe_sales.csv"
sales = pd.read_csv(url)
sales.head()

## Classification Empirical Study

## Conclusion

## References